# Example 5:  Modelling of an oxidation experiment

This benchmark describes the simulation of an oxidation experiment with marine pyrite-containing sediments. This modelling study was, together with the experimental work, originally reported by Appelo et al. (1998). The conceptual hydrochemical model and the implemented reaction network proposed for this experiment consists of a complex set of reactive processes. This includes

- the oxidation of pyrite, which is the primary driver of hydrochemical changes
- secondary reactions, such as dissolution of calcite, CO2 sorption, cation and proton exchange and
- oxidation of organic matter, a reaction that competes for the oxidation capacity supplied by the inflow solution. 

The modelling study considers three distinct phases of the experiment. In the first part the sediment material collected in the field was equilibrated with a 280 mmol MgCl<sub>2</sub> solution. In this phase the pore space becomes completely filled with the MgCl<sub>2</sub> solution and the exchangers sites are filled with Mg. In the second phase the column was fed with a more dilute MgCl<sub>2</sub> solution, while in the third phase the column was flushed for 4 pore volumes (at the same flow rate) with a hydrogen peroxide (H<sub>2</sub>O<sub>2</sub>) containing oxidising solution. Particularely the second phase provided the data that allowed to characterise the non-reactive transport  behaviour.

In [ ]:
from pathlib import Path
import os
from modflowapi.extensions import ApiSimulation
from modflowapi import Callbacks
# from workflow import *
from datetime import datetime
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#add mf6rtm path to the system
import flopy
from mf6rtm import utils, mf6rtm, mup3d
import re
import difflib

prefix = 'ex5'
DT_FMT = "%Y-%m-%d %H:%M:%S"
dataws = os.path.join("data")
databasews = os.path.join("database")


#stuff to compare outputs from pht3d and mf6rtm

def find_closest_match(query, dictionary):
    closest_match = difflib.get_close_matches(query, dictionary.keys(), n=1)
    if closest_match:
        return closest_match[0]
    else:
        return None
    
def calc_rows_from_ncol(variables, ncols=4):
    '''
    Calculates number of rows for subplots
    from ncols and len of variables to plot.

    Parameters:
        variables (list or sequence): list of variables to plot
        ncols (int): number of columns to plot
    '''
    n_subplots = len(variables)
    # calculate number of rows
    nrows = n_subplots // ncols + (n_subplots % ncols > 0)
    return nrows

## Flow and Transport Setup

In [ ]:
### Model params and setup

# General
length_units = "meters"
time_units = "days"

# Model discretization
nlay = 1  # Number of layers
Lx = 0.053 #m
ncol = 16 # Number of columns
nrow = 1  # Number of rows
delr = Lx/ncol #10.0  # Column width ($m$)
delc = 1 # Row width ($m$)
top = 2.87433E-03  # Top of the model ($m$)
# botm = 0.0  # Layer bottom elevations ($m$)
zbotm = 0.
botm = np.linspace(top, zbotm, nlay + 1)[1:]

#tdis
nper = 2  # Number of periods
nstp = [64, 100]  # Number of time steps
# nstp = [i*10 for i in nstp]
perlen = [ 0.9333, 1.45833]  # Simulation time ($days$)#100.0
# dt0 = perlen / nstp
tsmult = [1.0, 1.0]  # Time step multiplier
tdis_rc = [(kper, kstep, ts) for kper, kstep, ts in zip(perlen, nstp, tsmult)]

#injection
q = 2.4e-4 #injection rate m3/d
wel_spd = {i: [[(0,0,0), q]] for i in range(0, len(perlen))}


#hydraulic properties
prsity = 0.376 # Porosity
k11 = 1.0  # Horizontal hydraulic conductivity ($m/d$)
k33 = k11  # Vertical hydraulic conductivity ($m/d$)
strt = np.ones((nlay, nrow, ncol), dtype=float)*1
# two chd one for tailings and conc and other one for hds 

# two chd one for tailings and conc and other one for hds 
r_hd = 1
strt = np.ones((nlay, nrow, ncol), dtype=float)

chdspd = [[(i, 0, ncol-1), r_hd] for i in range(nlay)] # Constant head boundary $m$


#transport
dispersivity = 0.00537 #7.5e-5 Longitudinal dispersivity ($m$)
disp_tr_vert = dispersivity*0.01 # Transverse vertical dispersivity ($m$)
disp_tr_hor = dispersivity*0.01 # Transverse horizontal dispersivity ($m$)
diffc = 8.64e-4

icelltype = 1  # Cell conversion type

# Set solver parameter values (and related)
nouter, ninner = 300, 600
hclose, rclose, relax = 1e-6, 1e-6, 1.0

In [ ]:
perlen[0]/64, perlen[1]/100
perlen[0]

## Initialize Chemistry in Domain

In [ ]:
files = [f for f in os.listdir(dataws) if f.startswith(prefix)]

solutionsdf = pd.read_csv(os.path.join(dataws,f"{prefix}_solutions.csv"), comment = '#',  index_col = 0)

solutions = utils.solution_df_to_dict(solutionsdf)
solutions
# #assign solutions to grid
sol_ic = np.ones((nlay, nrow, ncol), dtype=float)
# sol_ic = 1
#add solutions to clss
solution = mup3d.Solutions(solutions)
solution.set_ic(sol_ic)

#exchange
excdf = pd.read_csv(os.path.join(dataws,f"{prefix}_exchange.csv"), comment = '#',  index_col = 0)
exchangerdic = utils.solution_df_to_dict(excdf)

exchanger = mup3d.ExchangePhases(exchangerdic)
exchanger_ic = np.ones((nlay, nrow, ncol), dtype=float)
exchanger_ic[0,0,:4] = 1
exchanger_ic[0,0,4:8] = 2
exchanger_ic[0,0,8:12] = 3
exchanger_ic[0,0,12:] = 4


exchanger.set_ic(exchanger_ic)
eq_solutions = [1,1,1,1]
exchanger.set_equilibrate_solutions(eq_solutions)

#kinetics
kinedic = utils.kinetics_phases_csv_to_dict(os.path.join(dataws,f"{prefix}_kinetic_phases.csv"))
orgsed_form = 'Orgc_sed -1.0 C 1.0' 
kinedic[1]['Orgc_sed'].append(orgsed_form)
kinetics = mup3d.KineticPhases(kinedic)
kinetics.set_ic(1)

#eq phases
equilibriums = utils.equilibrium_phases_csv_to_dict(os.path.join(dataws,f"{prefix}_equilibrium_phases.csv"))
equilibriums = mup3d.EquilibriumPhases(equilibriums)
equilibriums.set_ic(1)

#surfaces
surfdic = utils.surfaces_csv_to_dict(os.path.join(dataws,f"{prefix}_surfaces.csv"))
surfaces = mup3d.Surfaces(surfdic)
surfaces.set_ic(1)
# surfaces.set_options(['no_edl'])

#create model class
model = mup3d.Mup3d(prefix,solution, nlay, nrow, ncol)

#set model workspace
model.set_wd(os.path.join(f'{prefix}', f'mf6rtm'))

# #set database
database = os.path.join(databasews, f'ex5.dat')
model.set_database(database)


model.set_initial_temp([7., 7., 7.])
# #get postfix file
postfix = os.path.join(dataws, f'{prefix}_postfix.phqr')
model.set_postfix(postfix)

model.set_exchange_phases(exchanger)
model.set_phases(kinetics)
model.set_phases(equilibriums)
model.set_phases(surfaces)


In [ ]:
model.initialize()


In [ ]:
fixed_components = ['Orgc_sed']
# model.set_fixed_components(fixed_components)

In [ ]:
wellchem = mup3d.ChemStress('wel')
sol_spd = [2,3]
sol_spd
wellchem.set_spd(sol_spd)
model.set_chem_stress(wellchem)
model.wel.data

In [ ]:
model.sconc

In [ ]:
for key in wel_spd.keys():
    for i in range(len(wel_spd[key])):
        print(i)
        wel_spd[key][i].extend(model.wel.data[key])
wel_spd

In [ ]:
model.components

In [ ]:
def build_mf6_1d_injection_model(mup3d, nper, tdis_rc, length_units, time_units, nlay, nrow, ncol, delr, delc,
                                 top, botm, wel_spd, chdspd, prsity, k11, k33, dispersivity, icelltype, hclose, 
                                 strt, rclose, relax, nouter, ninner):

    #####################        GWF model           #####################
    gwfname = 'gwf'
    sim_ws = mup3d.wd
    sim = flopy.mf6.MFSimulation(sim_name=mup3d.name, sim_ws=sim_ws, exe_name='mf6')

    # Instantiating MODFLOW 6 time discretization
    flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_rc, time_units=time_units)

    # Instantiating MODFLOW 6 groundwater flow model
    gwf = flopy.mf6.ModflowGwf(
        sim,
        modelname=gwfname,
        save_flows=True,
        model_nam_file=f"{gwfname}.nam",
    )

    # Instantiating MODFLOW 6 solver for flow model
    imsgwf = flopy.mf6.ModflowIms(
        sim,
        complexity="complex",
        print_option="SUMMARY",
        outer_dvclose=hclose,
        outer_maximum=nouter,
        under_relaxation="NONE",
        inner_maximum=ninner,
        inner_dvclose=hclose,
        rcloserecord=rclose,
        linear_acceleration="CG",
        scaling_method="NONE",
        reordering_method="NONE",
        relaxation_factor=relax,
        filename=f"{gwfname}.ims",
    )
    sim.register_ims_package(imsgwf, [gwf.name])

    # Instantiating MODFLOW 6 discretization package
    dis = flopy.mf6.ModflowGwfdis(
        gwf,
        length_units=length_units,
        nlay=nlay,
        nrow=nrow,
        ncol=ncol,
        delr=delr,
        delc=delc,
        top=top,
        botm=botm,
        idomain=np.ones((nlay, nrow, ncol), dtype=int),
        filename=f"{gwfname}.dis",
    )
    dis.set_all_data_external()

    # Instantiating MODFLOW 6 node-property flow package
    npf = flopy.mf6.ModflowGwfnpf(
        gwf,
        save_flows=True,
        save_saturation = True,
        icelltype=icelltype,
        k=k11,
        k33=k33,
        save_specific_discharge=True,
        filename=f"{gwfname}.npf",
    )
    npf.set_all_data_external()
    # sto = flopy.mf6.ModflowGwfsto(gwf, ss=1e-6, sy=0.25)

    # Instantiating MODFLOW 6 initial conditions package for flow model
    flopy.mf6.ModflowGwfic(gwf, strt=strt, filename=f"{gwfname}.ic")
    
    wel = flopy.mf6.ModflowGwfwel(
            gwf,
            stress_period_data=wel_spd,
            save_flows = True,
            auxiliary = mup3d.components,
            pname = 'wel',
            filename=f"{gwfname}.wel"
        )
    wel.set_all_data_external()

    # Instantiating MODFLOW 6 constant head package
    chd = flopy.mf6.ModflowGwfchd(
        gwf,
        maxbound=len(chdspd),
        stress_period_data=chdspd,
        # auxiliary=mup3d.components,
        save_flows=False,
        pname="CHD",
        filename=f"{gwfname}.chd",
    )
    chd.set_all_data_external()

    # Instantiating MODFLOW 6 output control package for flow model
    oc_gwf = flopy.mf6.ModflowGwfoc(
        gwf,
        head_filerecord=f"{gwfname}.hds",
        budget_filerecord=f"{gwfname}.cbb",
        headprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
        saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
        printrecord=[("HEAD", "LAST"), ("BUDGET", "LAST")],
    )
    
    #####################           GWT model          #####################
    for c in mup3d.components:
        print(f'Setting model for component: {c}')
        gwtname = c
        
        # Instantiating MODFLOW 6 groundwater transport package
        gwt = flopy.mf6.MFModel(
            sim,
            model_type="gwt6",
            modelname=gwtname,
            model_nam_file=f"{gwtname}.nam"
        )

        # create iterative model solution and register the gwt model with it
        print('--- Building IMS package ---')
        imsgwt = flopy.mf6.ModflowIms(
            sim,
            # complexity="complex",
            print_option="SUMMARY",
            outer_dvclose=hclose,
            outer_maximum=nouter,
            under_relaxation="NONE",
            inner_maximum=ninner,
            inner_dvclose=hclose,
            rcloserecord=rclose,
            linear_acceleration="BICGSTAB",
            scaling_method="NONE",
            reordering_method="NONE",
            relaxation_factor=relax,
            filename=f"{gwtname}.ims",
        )
        sim.register_ims_package(imsgwt, [gwt.name])

        print('--- Building DIS package ---')
        dis = gwf.dis

        # create grid object
        dis = flopy.mf6.ModflowGwtdis(
            gwt,
            length_units=length_units,
            nlay=nlay,
            nrow=nrow,
            ncol=ncol,
            delr=delr,
            delc=delc,
            top=top,
            botm=botm,
            idomain=np.ones((nlay, nrow, ncol), dtype=int),
            filename=f"{gwtname}.dis",
        )
        dis.set_all_data_external()

         
        ic = flopy.mf6.ModflowGwtic(gwt, strt=mup3d.sconc[c], filename=f"{gwtname}.ic")
        ic.set_all_data_external()
        
        # Instantiating MODFLOW 6 transport source-sink mixing package
        sourcerecarray = ['wel', 'aux', f'{c}']
        # sourcerecarray = [()]
        ssm = flopy.mf6.ModflowGwtssm(
            gwt, 
            sources=sourcerecarray, 
            save_flows=True,
            print_flows=True,

            filename=f"{gwtname}.ssm"
        )
        ssm.set_all_data_external()
        # Instantiating MODFLOW 6 transport adv package
        print('--- Building ADV package ---')
        adv = flopy.mf6.ModflowGwtadv(
            gwt,
            scheme="tvd",
        )

        # Instantiating MODFLOW 6 transport dispersion package
        alpha_l = np.ones(shape=(nlay, nrow, ncol))*dispersivity  # Longitudinal dispersivity ($m$)
        ath1 = np.ones(shape=(nlay, nrow, ncol))*dispersivity*0.1 # Transverse horizontal dispersivity ($m$)
        atv = np.ones(shape=(nlay, nrow, ncol))*dispersivity*0.1   # Transverse vertical dispersivity ($m$)

        print('--- Building DSP package ---')
        dsp = flopy.mf6.ModflowGwtdsp(
            gwt,
            xt3d_off=True,
            alh=alpha_l,
            ath1=ath1,
            atv = atv,
            # diffc = diffc,
            filename=f"{gwtname}.dsp",
        )
        dsp.set_all_data_external()

        # Instantiating MODFLOW 6 transport mass storage package (formerly "reaction" package in MT3DMS)
        print('--- Building MST package ---')

        first_order_decay = None

        mst = flopy.mf6.ModflowGwtmst(
            gwt,
            porosity=prsity,
            first_order_decay=first_order_decay,
            filename=f"{gwtname}.mst",
        )
        mst.set_all_data_external()

        print('--- Building OC package ---')

        # Instantiating MODFLOW 6 transport output control package
        oc_gwt = flopy.mf6.ModflowGwtoc(
            gwt,
            budget_filerecord=f"{gwtname}.cbb",
            concentration_filerecord=f"{gwtname}.ucn",
            concentrationprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 10, "GENERAL")
                                        ],
            saverecord=[("CONCENTRATION", "ALL"), 
                        ("BUDGET", "ALL")
                        ],
            printrecord=[("CONCENTRATION", "ALL"), 
                            ("BUDGET", "ALL")
                            ],
        )

        # Instantiating MODFLOW 6 flow-transport exchange mechanism
        flopy.mf6.ModflowGwfgwt(
            sim,
            exgtype="GWF6-GWT6",
            exgmnamea=gwfname,
            exgmnameb=gwtname,
            filename=f"{gwtname}.gwfgwt",
        )

    sim.write_simulation()
    utils.prep_bins(sim_ws, src_path=os.path.join('..','bin'), get_only=['mf6', 'libmf6'])
    
    return sim

In [ ]:
sim = build_mf6_1d_injection_model(model, nper, tdis_rc, length_units, time_units, nlay, nrow, ncol, delr, delc,
                                    top, botm, wel_spd, chdspd, prsity, k11, k33, dispersivity, icelltype, hclose, 
                                    strt, rclose, relax, nouter, ninner)

In [ ]:
# mf6rtm.solve(model.wd)
model.run()

## Figures

In [ ]:
wd = os.path.join(f'{prefix}', f'pht3d')
dx = 0.01
simdf = pd.read_csv(os.path.join(wd, 'out.dat'), sep = '\t', skipinitialspace=True, index_col=[0])
simdf.drop(simdf.columns[len(simdf.columns)-1], axis=1, inplace=True)

simdf.loc[:, 'x'] = simdf['cell'] * delr

simapi = pd.read_csv(os.path.join(model.wd,'sout.csv'), sep = ',', skipinitialspace=True, index_col=[0])
simapi.loc[:, 'x'] = (simapi['cell'] + 1)*delr
simapi

# get all ucn files in wd

ucn_files = [f for f in os.listdir(wd) if f.lower().endswith('.ucn')]
ucn_files

# get file that ends in py
pht3dpy = [f for f in os.listdir(wd) if f.endswith('py')]


In [ ]:
#get apelo data
obs = pd.read_csv(os.path.join(dataws, 'ex5_obs.txt'), sep = '\t', skipinitialspace=True)
#replace -9999 with nan
obs.replace(-9999, np.nan, inplace=True)
#drop rows with all nan
obs.dropna(axis=0, how='all', inplace=True)

obs.columns =['vol', 'Mg', 'Ca', 'Alk', 'Cl', 'S(6)', 'pH']
obs.set_index('vol', inplace=True, drop=True)

obs = obs.iloc[3:, :]

In [ ]:
pncol=2
variables = simapi.iloc[:,1:-1].columns #dissolved only
#q from m3/d to ml*3/d

qml = q*1000000

pnrow = calc_rows_from_ncol(variables, pncol)

fig, axs = plt.subplots(pnrow,pncol, figsize = (10, 15))
for var, ax in zip(variables, axs.flatten()):
    # print(ucndic[var][:,-1,0,-1])
    # ax.plot(simdf.x.unique(), simdf.loc[perlen, var], label = 'Engesgaard1992 - PHT3D')
    df = simapi[simapi['cell'] == simapi['cell'].max()].copy()
    dfpht3d = simdf[simdf['cell'] == simdf['cell'].max()].copy()
    t = df.index
    # ax.plot([x for x in timespht3d], ucndic[var][:,0,0,0], label = f'{prefix} - PHT3D')
    ax.scatter(dfpht3d.index[::2]*qml, dfpht3d[var][::2], label = f'PHT3D', facecolors='none', edgecolors='r', s=15)
    ax.plot(t*qml, df.loc[:, var], label = f'MF6RTM', 
              zorder = 10)
    if var in obs.columns:
        ax.scatter(obs.index, obs[var], label = 'Appelo (1998)', color='g', s = 15, zorder = 1)
    #get min and max of y axis
    xmin, xmax = ax.get_ylim()
    ax.set_ylim(xmin*.8, xmax*1.2)

    ax.set_xlabel('outflow (ml)')
    if var not in ['pH', 'pe'] and len(var) < 4:
        ax.set_ylabel('C (mol L$^{-1}$)')
    elif var.startswith('si'):
        ax.set_ylabel(f'{var.upper()}')
    elif len(var) > 3:
        ax.set_ylabel(f'{var} (mol)')
    ax.set_title(f'{var}')
    ax.ticklabel_format(useOffset=False)
    ax.legend()
#get rid of last subplot
if len(variables) % pncol:
    fig.delaxes(axs.flatten()[-1])
fig.tight_layout()
fig.savefig(os.path.join(f'{prefix}.png'))